In [1]:
import pymanopt
import numpy as np

dim=3

In [2]:
man = pymanopt.manifolds.Sphere(dim)
A = np.random.rand(dim-1, dim-1)
cov = np.dot(A, A.transpose())
mean = man.random_point()

In [3]:
mean, cov

(array([-0.44328158, -0.46141816,  0.76850161]),
 array([[1.26839351, 0.32717228],
        [0.32717228, 0.13089927]]))

In [4]:
# generate tangent space
N = 100
samples = np.random.multivariate_normal(np.zeros(dim-1), cov, N)

In [5]:
# We need a tangent
normal = mean/np.linalg.norm(mean)

In [6]:
tol = 1e-5

def gram_schmidt(vectors):
    Q = np.zeros_like(vectors)
    for i in range(len(vectors)):
        v = vectors[i]
        for j in range(i):
            proj = np.dot(v, Q[j]) / np.dot(Q[j], Q[j]) * Q[j]
            v = v - proj
        Q[i] = v / np.linalg.norm(v)
    return Q
    
def get_axis(normal):
    # Assumes it is norm 1
    Q = np.eye(dim)
    arg = np.argmax(normal)
    if normal[arg]>(1-tol) and np.sum(normal)>(1-tol):
        return np.concatenate([Q[:arg], Q[arg+1:]])
    return gram_schmidt(np.vstack([normal,Q[:-1]]))[1:]

def get_e(i,n):
    return np.array(i*[0]+[1]+(n-i-1)*[0])

In [7]:
ax = get_axis(normal)

# Project points onto the axis to get all points in the tangent space
tangent_points = samples@ax

points = man.exp(mean, tangent_points)

In [11]:
np.linalg.det(cov)

np.float64(0.05899007853121325)

In [38]:
# For estimateing normalisation constant

def estimate_normalising(mu, Sigma, m, manifold, S=100):
    d = manifold.dim
    z = np.sqrt((2*np.pi)**d*np.linalg.det(Sigma))
    samples = np.random.multivariate_normal(np.zeros(d), Sigma, S)
    ax = get_axis(mu)
    vs = samples@ax
    return  z*np.mean(compute_vol(mu, vs, m, manifold))

sphere_M = lambda v: v

def compute_vol(mu, v, m, manifold):
    return np.sqrt(np.abs(m(manifold.exp(mu, v))))

In [39]:
estimate_normalising(mean, cov, sphere_M, man)

np.float64(0.897414545782442)

In [16]:
np.pi

3.141592653589793